In [1]:
from tensorflow.contrib.data import Dataset, Iterator
import pandas as pd
import numpy as np
import os
from daewoo_module import *
from tqdm import tqdm_notebook as tqdm

root_dir = ".\\input_data"
# img_dir = os.path.join(root_dir, '/figure'). 이미지는 os.path.join쓰면 안되는경우가 있음
img_dir = "./input_data/figure/"
logs_path = os.path.join(root_dir, "graph")

img = np.array([img_dir + x for x in os.listdir(img_dir)])
label = pd.read_csv(os.path.join(root_dir, 'description.csv'), engine='python')

# Classification과 regression 선택
classification = True

batch_size = 16
epochs = 2

if classification is True:
    label = pd.cut(label['WVHT ft.y'], bins=[0,5,10,100],labels=[0,1,2], include_lowest=True)
    label = np.array(label)
else:    
    label = label['WVHT ft.y'].values
    label = ((label - np.mean(label))/np.std(label)).reshape(-1,1)

C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_img_tensor, train_label_tensor, test_img_tensor, test_label_tensor = set_input(img, label)

train_imgs = Dataset.from_tensor_slices((train_img_tensor,train_label_tensor))
test_imgs = Dataset.from_tensor_slices((test_img_tensor,test_label_tensor))

if classification is True:
    train_imgs = train_imgs.map(input_tensor).batch(batch_size).shuffle(buffer_size=1000).repeat()
    test_imgs = test_imgs.map(input_tensor).batch(batch_size).shuffle(buffer_size=1000).repeat()
else:
    train_imgs = train_imgs.map(input_tensor_regression).batch(batch_size).shuffle(buffer_size=1000).repeat()
    test_imgs = test_imgs.map(input_tensor_regression).batch(batch_size).shuffle(buffer_size=1000).repeat()
    
train_iterator = train_imgs.make_initializable_iterator()
test_iterator = test_imgs.make_initializable_iterator()
handle = tf.placeholder(tf.string, shape=[])

iterator = tf.data.Iterator.from_string_handle(handle, train_imgs.output_types, train_imgs.output_shapes)
x, y = iterator.get_next()

train_batches = round(0.8 * len(label)) // batch_size
test_batches = (len(label)-round(0.8*len(label))) // batch_size

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [3]:
model_name = "VGG16_classification"
model = VGG16(x, y, bn=True, classification=classification)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))
    
    print("Training!")
    for i in tqdm(range(epochs)):
        print("-------{} Epoch--------".format(i+1))
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in tqdm(range(train_batches)):
            summary, _, acc, loss_ = sess.run([model.merged_summary_op, model.train,
                                               model.accuracy, model.loss],
                                        feed_dict={handle:train_handle,
                                                   model.learning_rate:0.001})
            if j % 10 == 0:
                print("Training Iter : {}, Acc : {.4f}, Loss : {:.4f}".format(j, acc, loss_))
                train_writer.add_summary(summary, j)
                summary, acc, loss_ = sess.run([model.merged_summary_op, model.accuracy,
                                                model.loss],
                                         feed_dict={handle:test_handle})
                print("Validation Iter : {}, Loss : {:.4f}".format(j, loss_))
                test_writer.add_summary(summary, j)
    
    print("-----------End of training-------------")
            

Training!


A Jupyter Widget

-------1 Epoch--------


A Jupyter Widget

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))
    
    print("Training!")
    for i in tqdm(range(epochs)):
        print("-------{} Epoch--------".format(i+1))
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in tqdm(range(train_batches)):
            summary, _, loss_ = sess.run([model.merged_summary_op, model.train, model.loss],
                                        feed_dict={handle:train_handle,
                                                   model.learning_rate:0.001})
            if j % 10 == 0:
                print("Training Iter : {}, Loss : {:.4f}".format(i+1, j, loss_))
                train_writer.add_summary(summary, j)
                summary, loss_ = sess.run([model.merged_summary_op, model.loss],
                                         feed_dict={handle:test_handle})
                print("Validation Iter : {}, Loss : {:.4f}".format(j, loss_))
                test_writer.add_summary(summary, j)
    
    print("-----------End of training-------------")
            
            

In [ ]:
model